In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer

In [2]:
df = pd.read_csv("Preprocessed.csv")

In [3]:
df.head()
asd = pd.DataFrame(df['AcceptedCmp6'].value_counts()).reset_index()
asd['percentage'] = (asd['count']/asd['count'].sum())*100
asd

,AcceptedCmp6,count,percentage
0,0,1734,84.792176
1,1,311,15.207824


### Our Baseline Accuracy should be above 84.79%
But we define our own profit function


In [6]:
X = df.drop(columns="AcceptedCmp6")
y = df["AcceptedCmp6"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [32]:
sale_revenue = 11
call_cost = 3
false_positive_cost = call_cost


def calculate_profit(y_true, y_pred):
    tn, fp, fn, tp= confusion_matrix(y_true, y_pred).ravel() #tn, fp, fn, tp
    profit_from_sales = tp * (sale_revenue - call_cost)
    loss_from_wasted_calls = fp * false_positive_cost
    total_profit = profit_from_sales - loss_from_wasted_calls

    return total_profit

def profit_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    return calculate_profit(y, y_pred)

scorer = make_scorer(profit_scorer, greater_is_better=True)




In [34]:
rfc = RandomForestClassifier(random_state=42, max_depth=4, class_weight='balanced', n_estimators= 50)
rfc.fit(X_train, y_train)
profit_scorer(rfc,X_train, y_test)

np.int64(841)

In [35]:
rf = RandomForestClassifier(random_state=42, class_weight="balanced")

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3,4,5],
    'criterion': ['gini', 'entropy'],        
    'max_features': ['sqrt', 'log2'],      
    'bootstrap': [True, False]              
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring= scorer)
grid_search.fit(X, y)


c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                     

KeyboardInterrupt: 